In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 20.3MB/s 
     |████████████████████████████████| 3.3MB 55.4MB/s 
     |████████████████████████████████| 901kB 50.1MB/s 


In [ ]:
import transformers

In [ ]:
print (transformers.__version__)

4.5.1


In [ ]:
import logging
import os
import math
import torch
import tensorflow as tf
from dataclasses import dataclass, field
from transformers import AutoModel, AutoTokenizer, BertTokenizerFast, BertForMaskedLM, BertModel
from transformers import TrainingArguments, HfArgumentParser
from transformers import LongformerModel 
from transformers import LongformerSelfAttention
from transformers import LongformerConfig 

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [ ]:
class BertLongSelfAttention(LongformerSelfAttention):
    def forward(
        self,
        hidden_states,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        past_key_value=None,
        output_attentions=False,
    ):
        return super().forward(hidden_states, attention_mask=attention_mask, output_attentions=output_attentions)


class BertLong(BertModel):
    def __init__(self, config):
        super().__init__(config)
        for i, layer in enumerate(self.encoder.layer):
            # replace the `modeling_bert.BertSelfAttention` object with `LongformerSelfAttention`
            layer.attention.self = BertLongSelfAttention(config, layer_id=i)

In [ ]:
def create_long_model(save_model_to, attention_window, max_pos):
    model = BertModel.from_pretrained("GroNLP/bert-base-dutch-cased")
    tokenizer = BertTokenizerFast.from_pretrained("GroNLP/bert-base-dutch-cased", model_max_length=max_pos)
    config = model.config

    print(max_pos)
    # extend position embeddings
    tokenizer.model_max_length = max_pos
    tokenizer.init_kwargs['model_max_length'] = max_pos
    current_max_pos, embed_size = model.embeddings.position_embeddings.weight.shape
    config.max_position_embeddings = max_pos
    assert max_pos > current_max_pos
    # allocate a larger position embedding matrix
    new_pos_embed = model.embeddings.position_embeddings.weight.new_empty(max_pos, embed_size)
    print(new_pos_embed.shape)
    print(model.embeddings.position_embeddings)
    # copy position embeddings over and over to initialize the new position embeddings
    k = 0
    step = current_max_pos
    while k < max_pos - 1:
        new_pos_embed[k:(k + step)] = model.embeddings.position_embeddings.weight
        k += step
    print(new_pos_embed.shape)
    model.embeddings.position_ids = torch.from_numpy(tf.range(new_pos_embed.shape[0], dtype=tf.int32).numpy()[tf.newaxis, :])
    model.embeddings.position_embeddings = torch.nn.Embedding.from_pretrained(new_pos_embed)
    
    # replace the `modeling_bert.BertSelfAttention` object with `LongformerSelfAttention`
    config.attention_window = [attention_window] * config.num_hidden_layers                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
    for i, layer in enumerate(model.encoder.layer):
        longformer_self_attn = LongformerSelfAttention(config, layer_id=i)
        longformer_self_attn.query = layer.attention.self.query
        longformer_self_attn.key = layer.attention.self.key
        longformer_self_attn.value = layer.attention.self.value

        longformer_self_attn.query_global = layer.attention.self.query
        longformer_self_attn.key_global = layer.attention.self.key
        longformer_self_attn.value_global = layer.attention.self.value

        layer.attention.self = longformer_self_attn
    print(model.embeddings.position_ids.shape)
    logger.info(f'saving model to {save_model_to}')
    model.save_pretrained(save_model_to)
    tokenizer.save_pretrained(save_model_to)
    return model, tokenizer, new_pos_embed

In [ ]:
@dataclass
class ModelArgs:
    attention_window: int = field(default=512, metadata={"help": "Size of attention window"})
    max_pos: int = field(default=4096, metadata={"help": "Maximum position"})

parser = HfArgumentParser((TrainingArguments, ModelArgs,))


training_args, model_args = parser.parse_args_into_dataclasses(look_for_args_file=False, args=[
    '--output_dir', 'tmp',
    '--warmup_steps', '500',
    '--learning_rate', '0.00003',
    '--weight_decay', '0.01',
    '--adam_epsilon', '1e-6',
    '--max_steps', '3000',
    '--logging_steps', '500',
    '--save_steps', '500',
    '--max_grad_norm', '5.0',
    '--per_gpu_eval_batch_size', '8',
    '--per_gpu_train_batch_size', '2',  # 32GB gpu with fp32
    '--gradient_accumulation_steps', '32',
    #'--evaluate_during_training',
    '--do_train',
    '--do_eval',
])

# Choose GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
model_path = f'{training_args.output_dir}/GroNLP/bert-base-dutch-cased-{model_args.max_pos}'
if not os.path.exists(model_path):
    os.makedirs(model_path)

logger.info(f'GroNLP/bert-base-dutch-cased into GroNLP/bert-base-dutch-cased-{model_args.max_pos}')
model, tokenizer, new_pos_embed = create_long_model(
    save_model_to=model_path, attention_window=model_args.attention_window, max_pos=model_args.max_pos)
#create_long_model(save_model_to, attention_window, max_pos)

INFO:__main__:GroNLP/bert-base-dutch-cased into GroNLP/bert-base-dutch-cased-4096
INFO:filelock:Lock 140210913015760 acquired on /root/.cache/huggingface/transformers/443c1d513d458927e5883e0b1298cdb70ba4d14a55faa236d93e0598efc78fc7.9c2c2a3ca9723b5324fb9d01fbe78b3550de8641c6cdf59498152ab427df95a2.lock


INFO:filelock:Lock 140210913015760 released on /root/.cache/huggingface/transformers/443c1d513d458927e5883e0b1298cdb70ba4d14a55faa236d93e0598efc78fc7.9c2c2a3ca9723b5324fb9d01fbe78b3550de8641c6cdf59498152ab427df95a2.lock
INFO:filelock:Lock 140211926223760 acquired on /root/.cache/huggingface/transformers/0afca22dbb0c0cf270ab24011224d449d604dca5dac95f18ef719be45029ee80.99bc8efd927a7d6b36398f23af86aff6b91a6b607e37773ed52ab62d53cfe5a9.lock


INFO:filelock:Lock 140211926223760 released on /root/.cache/huggingface/transformers/0afca22dbb0c0cf270ab24011224d449d604dca5dac95f18ef719be45029ee80.99bc8efd927a7d6b36398f23af86aff6b91a6b607e37773ed52ab62d53cfe5a9.lock


Some weights of BertModel were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:filelock:Lock 140210912984976 acquired on /root/.cache/huggingface/transformers/5ab1fe3a4445d1380cdcfca1496fda3de97bc83e5005fd1a0942cb758eacc67a.4c3119f5f60fe4102b00f9d33781542a23e127fb2ee1ece1254b8f908cc357bd.lock


INFO:filelock:Lock 140210912984976 released on /root/.cache/huggingface/transformers/5ab1fe3a4445d1380cdcfca1496fda3de97bc83e5005fd1a0942cb758eacc67a.4c3119f5f60fe4102b00f9d33781542a23e127fb2ee1ece1254b8f908cc357bd.lock
INFO:filelock:Lock 140210778477840 acquired on /root/.cache/huggingface/transformers/adb82a117c09b0f8768357de8e836a9e0610730782f82edc49dd0020c48f1d03.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock


INFO:filelock:Lock 140210778477840 released on /root/.cache/huggingface/transformers/adb82a117c09b0f8768357de8e836a9e0610730782f82edc49dd0020c48f1d03.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock
INFO:filelock:Lock 140210778848848 acquired on /root/.cache/huggingface/transformers/32589e144ae92d44c56eef5c89a01b3cf4bb1c744c60073f47834ddfdd84854e.fd86b436ab272b10c085aec068f5dbace040ca1602155bea390706b96cb608e3.lock


INFO:filelock:Lock 140210778848848 released on /root/.cache/huggingface/transformers/32589e144ae92d44c56eef5c89a01b3cf4bb1c744c60073f47834ddfdd84854e.fd86b436ab272b10c085aec068f5dbace040ca1602155bea390706b96cb608e3.lock



4096
torch.Size([4096, 768])
Embedding(512, 768)
torch.Size([4096, 768])


INFO:__main__:saving model to tmp/GroNLP/bert-base-dutch-cased-4096


torch.Size([1, 4096])


In [ ]:
logger.info(f'Loading the model from {model_path}')
tokenizer = BertTokenizerFast.from_pretrained(model_path)
model = BertLong.from_pretrained(model_path)


INFO:__main__:Loading the model from tmp/GroNLP/bert-base-dutch-cased-4096
